#**Notebook 5. Membaca Data Eksternal**

# Membaca File Dari Komputer Lokal

## Upload file dari komputer kita

Gunakan fitur `files.upload`

In [1]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('File yang diupload adalah "{name}" dengan panjang {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving annual-enterprise-survey-2021-financial-year-provisional-size-bands-csv.csv to annual-enterprise-survey-2021-financial-year-provisional-size-bands-csv.csv
File yang diupload adalah "annual-enterprise-survey-2021-financial-year-provisional-size-bands-csv.csv" dengan panjang 1492633 bytes


## Download file ke komputer kita

`files.download` merupakan fitur untuk mendownload file lewat browser ke komputer kita.


In [2]:
from google.colab import files

with open('contoh.txt', 'w') as f:
  f.write('Kalimat ini hanya sekedar contoh')

files.download('contoh.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Google Drive

Cara mengakses Google Drive ada 3 macam:
- Mounting Google Drive secara langsung selama running
- Menggunakan fungsi wrapper dengan API Google Drive misalnya [PyDrive](https://pythonhosted.org/PyDrive/)
- Menggunakan [native REST API](https://developers.google.com/drive/v3/web/about-sdk)



Perhatikan contohnya berikut ini.

## Mounting Google Drive

Contoh di bawah ini menunjukkan cara memasang Google Drive kita pada waktu proses menggunakan kode otorisasi, dan cara menulis dan membaca file di sana. Setelah dieksekusi, kita akan dapat melihat file baru (`contoh.txt`) at [https://drive.google.com/](https://drive.google.com/).

Ini hanya mendukung membaca, menulis, dan memindahkan file; untuk mengubah sharing setting atau metadata lainnya secara terprogram, gunakan salah satu opsi lain di bawah ini.

**Catatan:** Saat menggunakan tombol 'Mount Drive' di browser, kode autentikasi tidak diperlukan untuk notebook yang hanya diedit oleh pengguna saat ini.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
with open('/content/drive/My Drive/contoh.txt', 'w') as f:
  f.write('Halo semuanya selamat datang di Google Drive!')
!cat /content/drive/My\ Drive/foo.txt

cat: '/content/drive/My Drive/foo.txt': No such file or directory


In [5]:
drive.flush_and_unmount()
print('Semua perubahan yang dibuat dalam sesi Google Colaboratory ini sekarang akan terlihat di Google Drive.')

Semua perubahan yang dibuat dalam sesi Google Colaboratory ini sekarang akan terlihat di Google Drive.


## PyDrive

Contoh di bawah ini menunjukkan autentikasi dan unggah/unduh file menggunakan PyDrive. Contoh lebih lengkap tersedia di [PyDrive documentation](https://pythonhosted.org/PyDrive/).

In [6]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

Authenticate dan buat akun PyDrive client.


In [7]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Membuat dan mengupload file teks.


In [8]:
uploaded = drive.CreateFile({'title': 'Sample upload.txt'})
uploaded.SetContentString('Sample upload file content')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1lieOkcEB4I3gB6n8bCC4Djw3N9u6urUm


Memanggil file menurut ID dan menampilkannya.


In [9]:
downloaded = drive.CreateFile({'id': uploaded.get('id')})
print('Downloaded content "{}"'.format(downloaded.GetContentString()))

Downloaded content "Sample upload file content"


## Drive REST API

Untuk menggunakan Drive API, pertama-tama kita harus mengautentikasi dan membuat klien API.


In [10]:
from google.colab import auth
auth.authenticate_user()
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

Dengan klien ini, kita dapat menggunakan salah satu fungsi di [Google Drive API reference](https://developers.google.com/drive/v3/reference/). Perhatikan contoh berikut.


### Membuat file Drive baru dengan data dari Python

Pertama, buat file lokal untuk diunggah.

In [11]:
with open('/tmp/to_upload.txt', 'w') as f:
  f.write('my sample file')

print('/tmp/to_upload.txt contains:')
!cat /tmp/to_upload.txt

/tmp/to_upload.txt contains:
my sample file

Upload it using the [`files.create`](https://developers.google.com/drive/v3/reference/files/create) method. Further details on uploading files are available in the [developer documentation](https://developers.google.com/drive/v3/web/manage-uploads).

In [12]:
from googleapiclient.http import MediaFileUpload

file_metadata = {
  'name': 'Sample file',
  'mimeType': 'text/plain'
}
media = MediaFileUpload('/tmp/to_upload.txt',
                        mimetype='text/plain',
                        resumable=True)
created = drive_service.files().create(body=file_metadata,
                                       media_body=media,
                                       fields='id').execute()
print('File ID: {}'.format(created.get('id')))

File ID: 1Swo3MIeTgwqa79Kj3EaohV7ie0eV2gsK


After executing the cell above, you will see a new file named 'Sample file' at [https://drive.google.com/](https://drive.google.com/).

### Mengunduh data dari file Drive ke Python

Unduh file yang sudah kita upload di atas.

In [13]:
file_id = created.get('id')

import io
from googleapiclient.http import MediaIoBaseDownload

request = drive_service.files().get_media(fileId=file_id)
downloaded = io.BytesIO()
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
  # _ is a placeholder for a progress object that we ignore.
  # (Our file is small, so we skip reporting progress.)
  _, done = downloader.next_chunk()

downloaded.seek(0)
print('Downloaded file contents are: {}'.format(downloaded.read()))

Downloaded file contents are: b'my sample file'


In order to download a different file, set `file_id` above to the ID of that file, which will look like "1uBtlaggVyWshwcyP6kEI-y_W3P8D26sz".

# Google Sheets

Our examples below use the open-source [`gspread`](https://github.com/burnash/gspread) library for interacting with Google Sheets.

Import the library, authenticate, and create the interface to Sheets.

In [14]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

Below is a small set of `gspread` examples. Additional examples are available at the [`gspread` GitHub page](https://github.com/burnash/gspread#more-examples).

## Membuat sheet baru dengan data dari Python

In [15]:
sh = gc.create('My cool spreadsheet')

After executing the cell above, you will see a new spreadsheet named 'My cool spreadsheet' at [https://sheets.google.com](https://sheets.google.com/).

Open our new sheet and add some random data.

In [16]:
worksheet = gc.open('My cool spreadsheet').sheet1

cell_list = worksheet.range('A1:C2')

import random
for cell in cell_list:
  cell.value = random.randint(1, 10)

worksheet.update_cells(cell_list)

{'spreadsheetId': '1X-v3byjf-4dkQQ8uQ4_y7E0vfhD-oKEVWIxK6TYcpNc',
 'updatedRange': 'Sheet1!A1:C2',
 'updatedRows': 2,
 'updatedColumns': 3,
 'updatedCells': 6}

## Mengunduh data dari sheet ke Python sebagai Pandas DataFrame

Baca kembali data acak yang kita sudah masukkan di atas dan ubah hasilnya menjadi [Pandas DataFrame](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html).

In [17]:
worksheet = gc.open('My cool spreadsheet').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

import pandas as pd
pd.DataFrame.from_records(rows)

[['1', '4', '6'], ['6', '7', '3']]


,0,1,2
0,1,4,6
1,6,7,3
